# Estudio Comparativo Riguroso: Clasificacion Cuantica de Sonidos Respiratorios

## Clasificacion Binaria EPOC vs Sano mediante Aprendizaje Automatico Clasico y Cuantico

---

**Autor:** Alejandro Moris Lara  
**Asignatura:** Temas Avanzados de Ciencia e Ingenieria de Datos  
**Fecha:** Diciembre 2024

---
